In [1]:
import numpy as np
from scipy.signal import convolve2d, correlate2d

np.random.seed(42)

In [123]:
def sigmoid(x):
  return 1.0 / (1.0 + np.exp(-x))

def binary_cross_entropy(y_true, y_pred):
    return np.mean(-y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred))

def binary_cross_entropy_prime(y_true, y_pred):
    return ((1 - y_true) / (1 - y_pred) - y_true / y_pred) / np.size(y_true)

In [116]:
# Deklarasi input
learning_rate = 0.001
y = np.array([0])
A = np.array([
  [0.217, 0.232, 0.125],
  [0.132, 0.284, 0.202],
  [0.124, 0.222, 0.043]
])

In [148]:
# CNN - Layer 1, single filter, single bias, forward propagation
cnn1_b = np.array([
  [0.361, 0.518],
  [0.050, 0.108]
])
cnn1_k = np.array([
  [0.125, 0.607],
  [0.801, 0.613]
])

cnn1_1 = correlate2d(A, cnn1_k, mode='valid')
cnn1_1_b = cnn1_b + cnn1_1

print("Bias\n", cnn1_b)
print("Kernel\n", cnn1_k)
print("Convolution\n", cnn1_1)
print("Convolution with Bias\n", cnn1_1_b)


Bias
 [[0.361 0.518]
 [0.05  0.108]]
Kernel
 [[0.125 0.607]
 [0.801 0.613]]
Convolution
 [[0.447773 0.456185]
 [0.424298 0.362295]]
Convolution with Bias
 [[0.808773 0.974185]
 [0.474298 0.470295]]


In [201]:
# FCN - Layer 2, single weight, single bias, 1 unit, forward propagation
fcn_b = np.array([0.365])
fcn_w = np.array([0.909, 0.258, 0.311, 0.662])

fcn1 = np.matmul(fcn_w, cnn1_1_b.reshape(-1, 1))
fcn1_b = fcn_b + fcn1

fcn1_b_actived = sigmoid(fcn1_b.ravel())

print("Bias\n", fcn_b)
print("Weight\n", fcn_w)
print("Forward pass\n", fcn1)
print("Forward pass with Bias\n", fcn1_b)
print("Forward pass with bias and activation\n", fcn1_b_actived)

Bias
 [0.365]
Weight
 [0.909 0.258 0.311 0.662]
Forward pass
 [1.44535635]
Forward pass with Bias
 [1.81035635]
Forward pass with bias and activation
 [0.85940494]


In [188]:
# Empirical loss
E = binary_cross_entropy(y, fcn1_b_actived)
print(E)

1.961871417399307


In [237]:
# FCN - Layer 2, single weight, single bias, 1 unit, back propagation
fcn1_gradient = binary_cross_entropy_prime(y, fcn1_b_actived)

fcn1_weights_gradient = np.dot(fcn1_gradient, fcn1_b.reshape(-1, 1))
fcn1_input_gradient = np.dot(fcn_w.T.reshape(-1, 1), fcn1_gradient)

dfcn_w = learning_rate * fcn1_weights_gradient
dfcn_b = learning_rate * fcn1_gradient

print("Previous gradient\n", fcn1_gradient)
print("Gradient of weight\n", dfcn_w)
print("Gradient of bias\n", dfcn_b)
print("Gradient to backpropagate\n", fcn1_input_gradient)

Previous gradient
 [7.11262531]
Gradient of weight
 [0.01287639]
Gradient of bias
 [0.00711263]
Gradient to backpropagate
 [6.46537641 1.83505733 2.21202647 4.70855795]


In [234]:
# CNN - Layer 1, single filter, single bias, back propagation
cnn1_gradients = fcn1_input_gradient.reshape((2, 2))

kernels_gradient = correlate2d(A, cnn1_gradients, "valid")
cnn1_input_gradient = convolve2d(cnn1_gradients, cnn1_k, "full")

dkernels = learning_rate * kernels_gradient
dbiases = learning_rate * cnn1_gradients

print("Gradient of kernel\n", dkernels)
print("Gradient of bias\n", dbiases)  
print("Gradient to backpropagate\n", cnn1_input_gradient)

Gradient of kernel
 [[0.00345794 0.00330869]
 [0.00269418 0.00290039]]
Gradient of bias
 [[0.00646538 0.00183506]
 [0.00221203 0.00470856]]
Gradient to backpropagate
 [[0.80817205 4.15386564 1.1138798 ]
 [5.45526981 7.36442647 3.98298482]
 [1.7718332  5.12752715 2.88634603]]


In [247]:
kernels_gradient

array([[3.45793793, 3.30869372],
       [2.69417712, 2.90038635]])

## HELLWOE

In [260]:
a = np.array([
  [0.217, 0.232, 0.125],
  [0.132, 0.284, 0.202],
  [0.124, 0.222, 0.043]
])
b = np.array([
  [6.019, 1.708],
  [2.059, 4.383]
])

correlate2d(a, b, mode="valid")

0.17566666666666664

In [250]:
a = np.array([
  [6.019, 1.708],
  [2.059, 4.383]
])
b = np.array([
  [0.125, 0.607],
  [0.801, 0.613]
])
convolve2d(a, b, "full")

array([[0.752375, 3.867033, 1.036756],
       [5.078594, 6.855443, 3.707485],
       [1.649259, 4.77295 , 2.686779]])

In [261]:
a = np.array([
  [0.752, 3.867, 1.037],
  [5.079, 6.855, 3.707],
  [1.649, 4.772, 2.686]
])

np.mean(a)

3.378222222222222

In [265]:
a = 3.37823
b = np.array([
  [0.808, 0.974],
  [0.474, 0.470]
])

np.dot(a, b)

array([[2.72960984, 3.29039602],
       [1.60128102, 1.5877681 ]])